In [44]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn import svm
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

In [75]:
#file_name = "book_list_8_topics.csv"
#dimension = 8

file_name = "book_list_20_topics.csv"
dimension = 20

In [76]:
df = pd.read_csv(file_name)

In [77]:
dv = df[[str(i) for i in range(dimension)]].to_numpy()
cat = df["category"].to_numpy()

In [78]:
X_train, X_test, y_train, y_test = train_test_split(dv, cat, test_size=0.33, random_state=12)

def run_NB(X_train, X_test, y_train, y_test):
    text_clf = Pipeline([
                      ('clf', MultinomialNB())     #ML Model
                    ])
    text_clf.fit(X_train,y_train )
    predicted = text_clf.predict(X_test)
    print(metrics.confusion_matrix(y_test, predicted))
    print("NB:",np.mean(predicted == y_test) )
    
def run_DT(X_train, X_test, y_train, y_test):
    text_clf = Pipeline([
                          ('clf', tree.DecisionTreeClassifier())
                        ])
    clf = text_clf.fit(X_train, y_train) 

    predicted = clf.predict(X_test)

    print(metrics.confusion_matrix(y_test, predicted))
    print("DT:",np.mean(predicted == y_test) )
    
def run_SVM(X_train, X_test, y_train, y_test):
    
    text_clf = Pipeline([
                          ('clf', svm.LinearSVC(C=1.0))
                        ])
    text_clf.fit(X_train, y_train) 

    predicted = text_clf.predict(X_test)

    print(metrics.confusion_matrix(y_test, predicted))
    print(np.mean(predicted == y_test) )
    print(metrics.classification_report(y_test, predicted))

In [50]:
run_NB(X_train, X_test, y_train, y_test)
run_DT(X_train, X_test, y_train, y_test)
run_SVM(X_train, X_test, y_train, y_test)


[[  0   5   0   0   0   1   0   0   0]
 [  0 561   0   0   0  13   0   0   0]
 [  0  29   0   0   0   7   0   0   0]
 [  0  86   0   0   0   1   0   0   0]
 [  0 137   0   0   0  19   0   0   0]
 [  0 454   0   0   0  27   0   0   0]
 [  0 156   0   0   0   5   0   0   0]
 [  0  51   0   0   0   4   0   0   0]
 [  0 245   0   0   0  22   0   0   0]]
NB: 0.32254525507405374
[[  0   2   0   0   1   0   3   0   0]
 [ 11 149  13  53  21 194  42  10  81]
 [  0   8   0   3   0  23   0   0   2]
 [  1  60   3   5   2   2   1   4   9]
 [  0  18   3   3   7  79  41   0   5]
 [  2 216  41  12  63  59  12  30  46]
 [ 10  46   6   2  28  12   3  10  44]
 [  0  14   1   4   1  26   4   3   2]
 [  1 100  10   8  14  45  46   0  43]]
DT: 0.14755896873285793
[[  0   0   0   0   0   6   0   0   0]
 [  0 357   0   0   0 190   0   0  27]
 [  0  18   0   0   0  15   1   0   2]
 [  0  69   0   0   0  14   0   0   4]
 [  0  45   0   0   0  90   0   0  21]
 [  0 201   0   0   1 244   1   0  34]
 [  0  58   0 

d:\tech\mtech\plp\project\classifier\env\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [79]:
class_mapping = {
    "adventure": 0,
    "fantasy": 1,
    "historical": 2,
    "horror": 3,
    "mystery": 4,
    "romance": 5,
    "science-fiction": 6,
    "thriller": 7,
    "young-adult": 8
}
X_train, X_test, y_train, y_test = train_test_split(dv, cat, test_size=0.33, random_state=12)
y_train = np.vectorize(class_mapping.get)(y_train)
y_test = np.vectorize(class_mapping.get)(y_test)
print(y_train.shape)

(3699,)


In [80]:
import tensorflow as tf
from tensorflow import keras as ks
from tensorflow.keras.utils import to_categorical
# Validating the TensorFlow version
print(tf.__version__)

AUTOTUNE = tf.data.experimental.AUTOTUNE

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print(X_train.shape)
print(y_train.shape)

2.3.1
(3699, 20)
(3699, 9)


In [89]:
i = tf.keras.Input(shape=(20))
x = ks.layers.Dense(64, activation='relu')(i)
x = ks.layers.Dense(128, activation='relu')(x)
x = ks.layers.Dense(128, activation='relu')(x)
x = ks.layers.Dense(9, activation='sigmoid', name='prediction_layer')(x)
model = ks.Model(i, x)
model.summary()
model.compile(loss="CategoricalCrossentropy",optimizer='adam',metrics=['accuracy'])

Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 20)]              0         
_________________________________________________________________
dense_24 (Dense)             (None, 64)                1344      
_________________________________________________________________
dense_25 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_26 (Dense)             (None, 128)               16512     
_________________________________________________________________
prediction_layer (Dense)     (None, 9)                 1161      
Total params: 27,337
Trainable params: 27,337
Non-trainable params: 0
_________________________________________________________________


In [90]:
print(y_train)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200)

[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Epoch 1/200
116/116 [==============================] - 1s 4ms/step - loss: 1.8883 - accuracy: 0.2995 - val_loss: 1.7625 - val_accuracy: 0.3149
Epoch 2/200
116/116 [==============================] - 1s 4ms/step - loss: 1.7764 - accuracy: 0.3001 - val_loss: 1.7246 - val_accuracy: 0.3149
Epoch 3/200
116/116 [==============================] - 0s 3ms/step - loss: 1.7371 - accuracy: 0.3020 - val_loss: 1.6816 - val_accuracy: 0.3346
Epoch 4/200
116/116 [==============================] - 0s 4ms/step - loss: 1.6991 - accuracy: 0.3331 - val_loss: 1.6511 - val_accuracy: 0.3434
Epoch 5/200
116/116 [==============================] - 0s 4ms/step - loss: 1.6793 - accuracy: 0.3544 - val_loss: 1.6458 - val_accuracy: 0.3522
Epoch 6/200
116/116 [==============================] - 0s 4ms/step - loss: 1.6676 - accuracy: 0.3501 - val_loss: 1.6502 - val_acc

Epoch 57/200
116/116 [==============================] - 0s 3ms/step - loss: 1.4497 - accuracy: 0.4104 - val_loss: 1.6770 - val_accuracy: 0.3472
Epoch 58/200
116/116 [==============================] - 0s 4ms/step - loss: 1.4422 - accuracy: 0.4147 - val_loss: 1.6874 - val_accuracy: 0.3615
Epoch 59/200
116/116 [==============================] - 0s 4ms/step - loss: 1.4421 - accuracy: 0.4131 - val_loss: 1.6703 - val_accuracy: 0.3483
Epoch 60/200
116/116 [==============================] - 0s 4ms/step - loss: 1.4371 - accuracy: 0.4066 - val_loss: 1.6914 - val_accuracy: 0.3522
Epoch 61/200
116/116 [==============================] - 0s 3ms/step - loss: 1.4340 - accuracy: 0.4128 - val_loss: 1.7079 - val_accuracy: 0.3467
Epoch 62/200
116/116 [==============================] - 0s 3ms/step - loss: 1.4281 - accuracy: 0.4161 - val_loss: 1.6933 - val_accuracy: 0.3582
Epoch 63/200
116/116 [==============================] - 0s 4ms/step - loss: 1.4256 - accuracy: 0.4196 - val_loss: 1.6877 - val_accuracy:

Epoch 114/200
116/116 [==============================] - 0s 3ms/step - loss: 1.2024 - accuracy: 0.4826 - val_loss: 1.8959 - val_accuracy: 0.3154
Epoch 115/200
116/116 [==============================] - 0s 4ms/step - loss: 1.1951 - accuracy: 0.4923 - val_loss: 1.8981 - val_accuracy: 0.2907
Epoch 116/200
116/116 [==============================] - 0s 4ms/step - loss: 1.1873 - accuracy: 0.4858 - val_loss: 1.9104 - val_accuracy: 0.2830
Epoch 117/200
116/116 [==============================] - 0s 3ms/step - loss: 1.1901 - accuracy: 0.4853 - val_loss: 1.9245 - val_accuracy: 0.2863
Epoch 118/200
116/116 [==============================] - 0s 4ms/step - loss: 1.1812 - accuracy: 0.4909 - val_loss: 1.9145 - val_accuracy: 0.2929
Epoch 119/200
116/116 [==============================] - 0s 3ms/step - loss: 1.1862 - accuracy: 0.4915 - val_loss: 1.9164 - val_accuracy: 0.2858
Epoch 120/200
116/116 [==============================] - 0s 4ms/step - loss: 1.1738 - accuracy: 0.4972 - val_loss: 1.9659 - val_ac

116/116 [==============================] - 0s 4ms/step - loss: 0.9746 - accuracy: 0.5504 - val_loss: 2.3375 - val_accuracy: 0.2425
Epoch 171/200
116/116 [==============================] - 0s 4ms/step - loss: 0.9645 - accuracy: 0.5583 - val_loss: 2.3120 - val_accuracy: 0.2348
Epoch 172/200
116/116 [==============================] - 0s 4ms/step - loss: 0.9443 - accuracy: 0.5788 - val_loss: 2.3766 - val_accuracy: 0.2474
Epoch 173/200
116/116 [==============================] - 0s 4ms/step - loss: 0.9495 - accuracy: 0.5666 - val_loss: 2.3370 - val_accuracy: 0.2392
Epoch 174/200
116/116 [==============================] - 0s 4ms/step - loss: 0.9582 - accuracy: 0.5615 - val_loss: 2.3709 - val_accuracy: 0.2534
Epoch 175/200
116/116 [==============================] - 0s 4ms/step - loss: 0.9478 - accuracy: 0.5602 - val_loss: 2.3595 - val_accuracy: 0.2452
Epoch 176/200
116/116 [==============================] - 0s 4ms/step - loss: 0.9423 - accuracy: 0.5715 - val_loss: 2.3964 - val_accuracy: 0.2436